In [5]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [3]:
directory = Path("./ODIR-5K_Training_Dataset")

i = 0
for item in os.listdir(directory):
    i += 1

print(i)

7000


In [4]:
directory = Path("./ODIR-5K_Testing_Images")

i = 0
for item in os.listdir(directory):
    i += 1

print(i)

1000


In [6]:
df = pd.DataFrame(pd.read_excel("ODIR-5K_Training_Annotations(Updated)_V2.xlsx"))

In [7]:
df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,4686,63,Male,4686_left.jpg,4686_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
3496,4688,42,Male,4688_left.jpg,4688_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3497,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,0,0,0,0,0
3498,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0


In [18]:
train, test = train_test_split(df, test_size=(500/3500), random_state=42, shuffle=True)

In [19]:
# determine the path where to save the train and test file
data_dir = Path('./datasplits')
train_path = Path(data_dir, 'train.csv')
test_path = Path(data_dir, 'validation.csv')

# save the train and test file
# again using the '\t' separator to create tab-separated-values files
train.to_csv(train_path, index=False)
test.to_csv(test_path, index=False)

In [20]:
dftraining = pd.read_csv(train_path)
dfvalidation = pd.read_csv(test_path)

In [21]:
len(dftraining.index)

3000

In [22]:
len(dfvalidation.index)

500